In [ ]:
!pip install gradio
!pip install transformers
!pip install python-docx
!pip install torch
!pip install sentencepiece
!pip install soundfile
!pip install pillow
!pip install datasets
!pip install pyarrow==14.0.1
!pip install librosa
!pip install soundfile
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 969.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/

In [ ]:
import gradio as gr
from transformers import pipeline
import docx

# Inisialisasi pipeline Hugging Face untuk summarization menggunakan T5
summarizer = pipeline('summarization', model='t5-large', tokenizer='t5-large')

# Fungsi untuk membaca file txt
def read_txt(file):
    with open(file.name, 'r') as f:
        return f.read()

# Fungsi untuk membaca file docx
def read_docx(file):
    doc = docx.Document(file.name)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Fungsi utama untuk menangani input file dan menghasilkan ringkasan
def summarize_file(file):
    if file.name.endswith('.txt'):
        text = read_txt(file)
    elif file.name.endswith('.docx'):
        text = read_docx(file)
    else:
        return "Unsupported file type. Please upload a .txt or .docx file."

    # Gunakan model T5 untuk meringkas teks
    prefix = "summarize: "
    text = prefix + text
    summary = summarizer(text, max_length=10000, min_length=100, do_sample=False)[0]['summary_text']

    return summary

# Antarmuka Gradio
iface = gr.Interface(fn=summarize_file,
                     inputs=gr.File(),
                     outputs="text",
                     title="T5 Document Summarization",
                     description="Upload a .txt or .docx file for summarization using T5")

iface.launch()


ModuleNotFoundError: No module named 'gradio'

In [ ]:
import gradio as gr
from transformers import pipeline
import docx
import math

# Inisialisasi pipeline Hugging Face untuk summarization menggunakan T5
summarizer = pipeline('summarization', model='t5-large', tokenizer='t5-large')

# Fungsi untuk membaca file txt
def read_txt(file):
    with open(file.name, 'r') as f:
        return f.read()

# Fungsi untuk membaca file docx
def read_docx(file):
    doc = docx.Document(file.name)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Fungsi untuk membagi teks menjadi bagian yang lebih kecil (chunking)
def split_text(text, max_chunk_size=2096):
    words = text.split()
    chunks = [' '.join(words[i:i + max_chunk_size]) for i in range(0, len(words), max_chunk_size)]
    return chunks

# Fungsi utama untuk menangani input file dan menghasilkan ringkasan
def summarize_file(file):
    if file.name.endswith('.txt'):
        text = read_txt(file)
    elif file.name.endswith('.docx'):
        text = read_docx(file)
    else:
        return "Unsupported file type. Please upload a .txt or .docx file."

    # Memastikan teks tidak terlalu panjang untuk model
    max_token_per_chunk = 512  # Batas token yang diizinkan
    chunks = split_text(text, max_chunk_size=max_token_per_chunk)

    summarized_chunks = []
    for chunk in chunks:
        # Tambahkan prefix untuk model T5
        prefix = "summarize: "
        chunk = prefix + chunk
        # Ringkas tiap bagian (chunk) teks
        summary = summarizer(chunk, max_length=1024, min_length=100, do_sample=False)[0]['summary_text']
        summarized_chunks.append(summary)

    # Gabungkan hasil ringkasan dari setiap bagian
    full_summary = ' '.join(summarized_chunks)

    return full_summary

# Antarmuka Gradio
iface = gr.Interface(fn=summarize_file,
                     inputs=gr.File(),
                     outputs="text",
                     title="T5 Document Summarization",
                     description="Upload a .txt or .docx file for summarization using T5")

iface.launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://06c0e2de9dacec3cf1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from transformers import pipeline
import docx

# Inisialisasi pipeline Hugging Face untuk summarization, image-to-text, audio-to-text, dan NLP conversation
summarizer = pipeline('summarization', model='t5-large', tokenizer='t5-large')
image_captioning = pipeline('image-to-text', model='Salesforce/blip-image-captioning-large')
audio_to_text = pipeline('automatic-speech-recognition', model='facebook/wav2vec2-large-960h')
conversation = pipeline('conversational', model='facebook/blenderbot-400M-distill')

# Fungsi untuk membaca file txt
def read_txt(file):
    with open(file.name, 'r') as f:
        return f.read()

# Fungsi untuk membaca file docx
def read_docx(file):
    doc = docx.Document(file.name)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Fungsi untuk membagi teks menjadi bagian yang lebih kecil (chunking)
def split_text(text, max_chunk_size=2096):
    words = text.split()
    chunks = [' '.join(words[i:i + max_chunk_size]) for i in range(0, len(words), max_chunk_size)]
    return chunks

# Fungsi utama untuk menangani input file dan menghasilkan ringkasan
def summarize_file(file):
    if file.name.endswith('.txt'):
        text = read_txt(file)
    elif file.name.endswith('.docx'):
        text = read_docx(file)
    else:
        return "Unsupported file type. Please upload a .txt or .docx file."

    # Memastikan teks tidak terlalu panjang untuk model
    max_token_per_chunk = 512  # Batas token yang diizinkan
    chunks = split_text(text, max_chunk_size=max_token_per_chunk)

    summarized_chunks = []
    for chunk in chunks:
        prefix = "summarize: "
        chunk = prefix + chunk
        summary = summarizer(chunk, max_length=1024, min_length=100, do_sample=False)[0]['summary_text']
        summarized_chunks.append(summary)

    full_summary = ' '.join(summarized_chunks)
    return full_summary

# Fungsi untuk image-to-text
def image_to_text(image):
    return image_captioning(image)[0]['generated_text']

# Fungsi untuk audio-to-text
def audio_to_text_conversion(audio):
    return audio_to_text(audio)['text']

# Fungsi untuk conversation
def conversation_fn(input_text):
    return conversation(input_text)[0]['generated_text']

# Antarmuka Gradio
iface = gr.Interface(
    fn=summarize_file,
    inputs=[
        gr.File(label="Upload .txt or .docx File"),     # Text summarization input
        gr.Image(label="Upload an Image"),             # Image to Text input
        gr.Audio(label="Upload an Audio file"),        # Audio to Text input
        gr.Textbox(label="Conversation Input")         # NLP conversation input
    ],
    outputs=[
        gr.Textbox(label="Summarized Text"),          # Output for text summarization
        gr.Textbox(label="Image Description"),        # Output for image captioning
        gr.Textbox(label="Transcribed Audio"),        # Output for audio-to-text
        gr.Textbox(label="Conversation Response")     # Output for NLP conversation
    ],
    title="Multimodal Summarization, Captioning, Transcription, and Conversation",
    description="Upload files for summarization, image-to-text, audio-to-text, or engage in a conversation.",
)

iface.launch()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

KeyError: "Unknown task conversational, available tasks are ['audio-classification', 'automatic-speech-recognition', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"

In [ ]:
import gradio as gr
from transformers import pipeline
import docx

# Inisialisasi pipeline Hugging Face untuk summarization, image-to-text, dan conversation
summarizer = pipeline('summarization', model='t5-large', tokenizer='t5-large')
image_captioner = pipeline('image-to-text', model='Salesforce/blip-image-captioning-large')
speech_to_text = pipeline('automatic-speech-recognition', model='openai/whisper-large')
conversation_model = pipeline('text-generation', model='meta-llama/Llama-3.1-8B-Instruct')

# Fungsi untuk membaca file txt
def read_txt(file):
    with open(file.name, 'r') as f:
        return f.read()

# Fungsi untuk membaca file docx
def read_docx(file):
    doc = docx.Document(file.name)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Fungsi untuk membagi teks menjadi bagian yang lebih kecil (chunking)
def split_text(text, max_chunk_size=2096):
    words = text.split()
    chunks = [' '.join(words[i:i + max_chunk_size]) for i in range(0, len(words), max_chunk_size)]
    return chunks

# Fungsi untuk ringkasan file
def summarize_file(file):
    if file.name.endswith('.txt'):
        text = read_txt(file)
    elif file.name.endswith('.docx'):
        text = read_docx(file)
    else:
        return "Unsupported file type. Please upload a .txt or .docx file."

    max_token_per_chunk = 512  # Batas token untuk model
    chunks = split_text(text, max_chunk_size=max_token_per_chunk)

    summarized_chunks = []
    for chunk in chunks:
        prefix = "summarize: "
        chunk = prefix + chunk
        summary = summarizer(chunk, max_length=1024, min_length=100, do_sample=False)[0]['summary_text']
        summarized_chunks.append(summary)

    full_summary = ' '.join(summarized_chunks)
    return full_summary

# Fungsi untuk image-to-text
def image_to_text(image):
    caption = image_captioner(image)[0]['generated_text']
    return caption

# Fungsi untuk audio-to-text
def audio_to_text(audio):
    transcription = speech_to_text(audio)['text']
    return transcription

# Fungsi untuk conversation dengan Llama
def conversation(text):
    response = conversation_model(text, max_length=100)[0]['generated_text']
    return response

# Antarmuka Gradio
iface = gr.Interface(
    fn={
        "summarize_file": summarize_file,
        "image_to_text": image_to_text,
        "audio_to_text": audio_to_text,
        "conversation": conversation
    },
    inputs=[
        gr.File(label="Upload a .txt or .docx file"),  # Summarization
        gr.Image(label="Upload an Image"),            # Image-to-Text
        gr.Audio(label="Upload an Audio File"),       # Audio-to-Text
        gr.Textbox(lines=2, label="Chat with NLP")    # Conversation
    ],
    outputs=[
        "text",  # Summarization output
        "text",  # Image caption output
        "text",  # Audio transcription output
        "text"   # NLP conversation output
    ],
    title="Multimodal Summarization and NLP Tool",
    description="Upload files for summarization, image-to-text, audio-to-text, or chat with the NLP model (Llama-3.1)."
)

iface.launch()


config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct.
401 Client Error. (Request ID: Root=1-66f45a17-38e84c666cc5e2cd154bc330;51039fcc-65f0-4261-83fb-27442a501d12)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
import gradio as gr
from transformers import pipeline
import docx

# Inisialisasi pipeline Hugging Face untuk summarization, image-to-text, audio-to-text, dan conversation
summarizer = pipeline('summarization', model='t5-large', tokenizer='t5-large')
image_captioner = pipeline('image-to-text', model='Salesforce/blip-image-captioning-large')
speech_to_text = pipeline('automatic-speech-recognition', model='openai/whisper-large')
conversation_model = pipeline('text2text-generation', model='google/flan-t5-large')  # Using Flan-T5 for conversation

# Fungsi untuk membaca file txt
def read_txt(file):
    with open(file.name, 'r') as f:
        return f.read()

# Fungsi untuk membaca file docx
def read_docx(file):
    doc = docx.Document(file.name)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Fungsi untuk membagi teks menjadi bagian yang lebih kecil (chunking)
def split_text(text, max_chunk_size=2096):
    words = text.split()
    chunks = [' '.join(words[i:i + max_chunk_size]) for i in range(0, len(words), max_chunk_size)]
    return chunks

# Fungsi untuk ringkasan file
def summarize_file(file):
    if file.name.endswith('.txt'):
        text = read_txt(file)
    elif file.name.endswith('.docx'):
        text = read_docx(file)
    else:
        return "Unsupported file type. Please upload a .txt or .docx file."

    max_token_per_chunk = 512  # Batas token untuk model
    chunks = split_text(text, max_chunk_size=max_token_per_chunk)

    summarized_chunks = []
    for chunk in chunks:
        prefix = "summarize: "
        chunk = prefix + chunk
        summary = summarizer(chunk, max_length=1024, min_length=100, do_sample=False)[0]['summary_text']
        summarized_chunks.append(summary)

    full_summary = ' '.join(summarized_chunks)
    return full_summary

# Fungsi untuk image-to-text
def image_to_text(image):
    caption = image_captioner(image)[0]['generated_text']
    return caption

# Fungsi untuk audio-to-text
def audio_to_text(audio):
    transcription = speech_to_text(audio)['text']
    return transcription

# Fungsi untuk conversation dengan Flan-T5
def conversation(text):
    response = conversation_model(text)[0]['generated_text']
    return response

# Antarmuka Gradio
iface = gr.Interface(
    fn=summarize_file, # Changed to a single function
    inputs=[
        gr.File(label="Upload a .txt or .docx file"),  # Summarization
        gr.Image(label="Upload an Image"),            # Image-to-Text
        gr.Audio(label="Upload an Audio File"),       # Audio-to-Text
        gr.Textbox(lines=2, label="Chat with NLP")    # Conversation
    ],
    outputs=[
        "text",  # Summarization output
        "text",  # Image caption output
        "text",  # Audio transcription output
        "text"   # NLP conversation output
    ],
    title="Multimodal Summarization and NLP Tool",
    description="Upload files for summarization, image-to-text, audio-to-text, or chat with the NLP model (Flan-T5)."
)
iface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1002: UserWarning: Expected 1 arguments for function <function summarize_file at 0x7c2124348700>, received 4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1010: UserWarning: Expected maximum 1 arguments for function <function summarize_file at 0x7c2124348700>, received 4.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://73adb6c4e6f4058b11.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from transformers import pipeline
import docx

# Initializing Hugging Face models
summarizer = pipeline('summarization', model='t5-large', tokenizer='t5-large')
image_captioner = pipeline('image-to-text', model='Salesforce/blip-image-captioning-large')
speech_to_text = pipeline('automatic-speech-recognition', model='openai/whisper-large')
conversation_model = pipeline('text2text-generation', model='google/flan-t5-large')  # Flan-T5 for conversation

# Function to read a .txt file
def read_txt(file):
    with open(file.name, 'r') as f:
        return f.read()

# Function to read a .docx file
def read_docx(file):
    doc = docx.Document(file.name)
    full_text = [para.text for para in doc.paragraphs]
    return '\n'.join(full_text)

# Function to handle summarization
def summarize_file(file):
    if file.name.endswith('.txt'):
        text = read_txt(file)
    elif file.name.endswith('.docx'):
        text = read_docx(file)
    else:
        return "Unsupported file type. Please upload a .txt or .docx file."

    # Splitting text into chunks if necessary (max 512 tokens per chunk)
    chunks = [text[i:i + 1024] for i in range(0, len(text), 1024)]

    summarized_chunks = [summarizer(chunk, max_length=1024, min_length=100, do_sample=False)[0]['summary_text'] for chunk in chunks]

    return ' '.join(summarized_chunks)

# Function to handle image-to-text
def image_to_text(image):
    caption = image_captioner(image)[0]['generated_text']
    return caption

# Function to handle audio-to-text
def audio_to_text(audio):
    transcription = speech_to_text(audio)['text']
    return transcription

# Function for conversation (chat-based interaction)
def conversation(text):
    response = conversation_model(text)[0]['generated_text']
    return response

# Main handler that routes to the correct function based on user input type
def process_input(choice, file=None, image=None, audio=None, text=None):
    if choice == "Summarize Document":
        if file is not None:
            return summarize_file(file)
        else:
            return "Please upload a .txt or .docx file."
    elif choice == "Image to Text":
        if image is not None:
            return image_to_text(image)
        else:
            return "Please upload an image."
    elif choice == "Audio to Text":
        if audio is not None:
            return audio_to_text(audio)
        else:
            return "Please upload an audio file."
    elif choice == "Conversation":
        if text is not None:
            return conversation(text)
        else:
            return "Please enter text for conversation."

# Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("# MODEL NLP UNIMODEL BY NOPALZ")

    with gr.Row():
        choice = gr.Radio(choices=["Summarize Document", "Image to Text", "Audio to Text", "Conversation"], label="Choose Input Type")

    file_input = gr.File(label="Upload Document (.txt or .docx)", visible=False)
    image_input = gr.Image(label="Upload Image", visible=False)
    audio_input = gr.Audio(label="Upload Audio", visible=False)
    text_input = gr.Textbox(label="Enter Text", visible=False)

    output_text = gr.Textbox(label="Output", lines=10)

    def update_input_fields(choice):
        if choice == "Summarize Document":
            return gr.update(visible=True), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
        elif choice == "Image to Text":
            return gr.update(visible=False), gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)
        elif choice == "Audio to Text":
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)
        elif choice == "Conversation":
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)

    choice.change(fn=update_input_fields, inputs=choice, outputs=[file_input, image_input, audio_input, text_input])

    submit_button = gr.Button("Submit")

    submit_button.click(
        fn=process_input,
        inputs=[choice, file_input, image_input, audio_input, text_input],
        outputs=output_text
    )

iface.launch()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4ed864b81700203967.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
